In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

In [11]:
seed = 0
# a = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
# b = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
# tensorflow, numpy 랜덤값을 설정한다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [12]:
# 학습할 데이터를 만든다. [공부시간, 과외시간]
# xData = [[2,3],[4,3],[6,4],[8,6],[10,7],[12,8],[14,9]]
xData = np.array([[2,3],[4,3],[6,4],[8,6],[10,7],[12,8],[14,9]]) # 7행 2열 numpy 배열
print(xData)

[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]


In [13]:
# yData -> [합격여부] -> 실제값
# yData = np.array([0,0,0,1,1,1,1]) # 1행 7열
# reshape() : numpy에서 데이터는 그대로 유지한 채 배열의 형태(차원)를 변경한다.
# 1행 7열 numpy 배열 -> 7행 1열 numpy 배열
yData = np.array([0,0,0,1,1,1,1]).reshape(7,1)
print(yData)

[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]


In [25]:
# xData와 yData를 저장할 placeholder를 만든다.
# placeholder에 numpy 배열이 대입되는 경우 shape 속성을 이용해 대입되는 numpy 배열의 차원을 지정해야 한다.
# [None, 2] -> numpy 행의 개수는 몇개라도 상관없고 열의 개수는 2개이다.
X = tf.placeholder(dtype=tf.float64, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float64, shape=[None, 1]) # yData가 대입될 placeholder

# 기울기(가중치) a와 y절편(바이어스) b의 값을 임의(random)로 정한다.
# xData 1건은 1행 2열인 행렬이므로 가중치 a를 2행 1열인 난수로 발생시키면 
# 행렬의 곱을 이용해 편리하게 연산할 수 있다.
a = tf.Variable(tf.random_uniform([2,1], dtype=tf.float64)) # 난수를 2행 1렬로 발생시킨다.
b = tf.Variable(tf.random_uniform([1], dtype=tf.float64)) 

# 시그모이드 방정식 만들기
y = tf.sigmoid(tf.matmul(X,a)+b)

# 오차 함수
loss = -tf.reduce_mean(Y*tf.log(y)+(1-Y)*tf.log(1-y))

# 오차를 최소화하는 경사하강 알고리즘
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [37]:
# sigmoid() 함수의 실행결과(y) 값이 0.5 이상이면 1을, 0.5 미만이면 0을 리턴시킨다.
# tf.cast(캐스팅할 데이터, dtype=캐스팅 될 데이터 타입) : tensorflow에서 형변환을 실행하는 메소드
# predicted = tf.cast(tf.constant([1.9,2.1]), dtype=tf.int32) 
# [1.2] -> 실수(부동소수점 데이터)를 정수형 데이터로 변경한다.
# predicted = tf.cast(0.4 >= 0.5, dtype=tf.float64) 
# 조건 비교 결과가 False이므로 0으로 캐스팅 한다.
# predicted = tf.cast(0.5 >= 0.5, dtype=tf.float64) 
# 조건 비교 결과가 True이므로 1로 캐스팅 한다.


predicted = tf.cast(y >= 0.5, dtype=tf.float64)

# 예상값이 실제값과 일치하는 정도(정확도)를 계산한다.
# tf.equal() : tensorflow에서 인수로 지정된 값이 같으면 True 다르면 False 결과를
# tf.float64로 캐스팅하고 실행된 전체 평균을 계산한다.

# 정확도
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float64)) 

In [39]:
# 학습을 시킨다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3001):
        # count : 학습횟수, a1: 기울기(공부시간), a2: 기울기(과외시간), b: y절편, loss: 오차함수
        a_, b_, loss_, _ = sess.run([a,b,loss,gradient_descent], feed_dict={X:xData,Y:yData})
#         if i % 300 == 0:
#             print('count = {}, a1 = {}, a2 = {}, b = {}, loss = {}'.format(i, a_[0], a_[1], b_, loss_))
# 학습 완료 -----------------------------------------------------------
# 테스트 데이터

    new_x = np.array([6,5]).reshape(1,2) # reshape() 함수를 이용해서 1행 2열인 2차원 numpy 배열을 만든다.

    # sigmoid() 메소드 연산 결과에 테스트 데이터를 대입시켜 연산한다.
    result, new_y = sess.run([predicted, y], feed_dict={X: new_x})
    print("공부시간 : %d, 과외시간 : %d" %(new_x[:,0], new_x[:,1]))
    print("합격여부(sigmoid() 메소드 실행결과를 0.5이상은 1, 미만은 0으로 변환한다.): %d, 합격확률(sigmoid() 함수 실행 결과 : %f)" %\
          (result, new_y))
    for i in range(11):
        new_x = np.array([0,i]).reshape(1,2)
        result, new_y = sess.run([predicted, y], feed_dict={X: new_x})
        print("공부시간:%d, 과외시간:%2d" %(new_x[:,0],new_x[:,1]), end=',')
        print("합격 여부 : %s, 합격확률: %8.4f%%" %('합격' if result==1 else'불합격', new_y*100))

공부시간 : 6, 과외시간 : 5
합격여부(sigmoid() 메소드 실행결과를 0.5이상은 1, 미만은 0으로 변환한다.): 1, 합격확률(sigmoid() 함수 실행 결과 : 0.512863)
공부시간:0, 과외시간: 0,합격 여부 : 불합격, 합격확률:   0.0133%
공부시간:0, 과외시간: 1,합격 여부 : 불합격, 합격확률:   0.0657%
공부시간:0, 과외시간: 2,합격 여부 : 불합격, 합격확률:   0.3227%
공부시간:0, 과외시간: 3,합격 여부 : 불합격, 합격확률:   1.5699%
공부시간:0, 과외시간: 4,합격 여부 : 불합격, 합격확률:   7.2847%
공부시간:0, 과외시간: 5,합격 여부 : 불합격, 합격확률:  27.9057%
공부시간:0, 과외시간: 6,합격 여부 : 합격, 합격확률:  65.5988%
공부시간:0, 과외시간: 7,합격 여부 : 합격, 합격확률:  90.3791%
공부시간:0, 과외시간: 8,합격 여부 : 합격, 합격확률:  97.8849%
공부시간:0, 과외시간: 9,합격 여부 : 합격, 합격확률:  99.5633%
공부시간:0, 과외시간:10,합격 여부 : 합격, 합격확률:  99.9110%
